## Import Modules

In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
from scipy.signal import savgol_filter
from pathlib import Path

DATA_DIR = Path('../data')

%matplotlib inline
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 20

sns.set_style("whitegrid",{"axes.edgecolor": "0",'grid.color': '0','font.family':'Arial'})
sns.set_context("notebook",font_scale=2)

## Load Data

In [2]:
# Really convoluted method of reading in data due to package clashes 

# Dataframe for lobedata
lobe_data_w_vinegar = pd.read_hdf(DATA_DIR / 'experiments_summary.h5')

# Numpy array for df_fs
file = h5py.File(DATA_DIR / 'experiments_summary.h5', 'r')
df_fs = file['df_fs'][:] # numpy ndarray

In [3]:
file.keys()

<KeysViewHDF5 ['df_fs', 'lobe_summary']>

In [4]:
# Collect data into a preliminary df
lobe_activities = np.row_stack(df_fs)
lobe_data_w_vinegar["df_f"] = np.mean(lobe_activities[:, 31:35], 1)
lobe_data_w_vinegar["ΔF/F"] = np.mean(lobe_activities[:, 31:35], 1)
lobe_data_w_vinegar["pre_ΔF/F"] = np.mean(np.abs(lobe_activities[:, 6:30]),1)

lobe_data_w_vinegar.head()

,experiment_id,stimulus,valence,dorsal_to_ventral,hours_starved,order_presented,virgin,lobe,adaptation,pre_std,single_pre_std,single_pre_amp,response_std,fly_id,fly_day_id,mated_single,df_mean,df_f,ΔF/F,pre_ΔF/F
0,TH-58E02_090816_Citronella,Citronella,-1.0,True,0,0,False,a1,0.007500,0.030948,0.170519,0.131845,0.181160,2016-08-09 00:00:000,0,NaN,0.341723,0.341723,0.341723,0.021723
1,TH-58E02_090816_Citronella,Citronella,-1.0,True,0,0,False,a2,-0.010356,0.020711,0.159267,0.217365,0.157602,2016-08-09 00:00:000,0,NaN,0.230774,0.230774,0.230774,0.017802
2,TH-58E02_090816_Citronella,Citronella,-1.0,True,0,0,False,a3,-0.027494,0.026490,0.115774,0.135316,0.343259,2016-08-09 00:00:000,0,NaN,1.023435,1.023435,1.023435,0.020086
3,TH-58E02_090816_Citronella,Citronella,-1.0,True,0,0,False,ap1,0.026010,0.041217,0.189557,0.139339,0.258084,2016-08-09 00:00:000,0,NaN,0.476658,0.476658,0.476658,0.028166
4,TH-58E02_090816_Citronella,Citronella,-1.0,True,0,0,False,ap2,0.000798,0.041212,0.239678,0.180607,0.268255,2016-08-09 00:00:000,0,NaN,0.412485,0.412485,0.412485,0.034686


### Refine the DF

In [5]:
lobe_data_w_vinegar = lobe_data_w_vinegar.replace("1-Heptanone",'2-Heptanone')

In [6]:
Lobes = lobe_data_w_vinegar.lobe.unique()
print(f'Lobes: {Lobes}')

LobesGreek=['α1', 'α2', 'α3', "α'1", "α'2", "α'3", 'β1', 'β2', "β'1", "β'2",
       'γ1', 'γ2', 'γ3', 'γ4', 'γ5']

dictionaryGreek = dict(zip(Lobes, LobesGreek))

lobe_data_w_vinegar["greek_lobe"] = [dictionaryGreek[l] for l in lobe_data_w_vinegar.lobe]

Lobes: ['a1' 'a2' 'a3' 'ap1' 'ap2' 'ap3' 'b1' 'b2' 'bp1' 'bp2' 'g1' 'g2' 'g3'
 'g4' 'g5']


In [7]:
# Add hours starved col
lobe_data_w_vinegar["hours starved"] = lobe_data_w_vinegar["hours_starved"]

In [8]:
# Add valence column (1, -1, np.nan)

valence_map = {
    "high_Vinegar": np.nan,
    "Vinegar": '1',
    "Yeast": '1',
    "Isoamylacetate": np.nan,
    "2-Heptanone": np.nan,
    "1-Heptanone": np.nan,
    "cVA": np.nan,
    "3Octanol": '-1',
    "Ethanol": np.nan,
    "1-Hexanol": np.nan,
    "Peppermint": '-1',
    "4MCH": '-1',
    "Citronella": '-1',
    "Geosmin": '-1',
}

lobe_data_w_vinegar["valence"] = [valence_map[s] for s in lobe_data_w_vinegar.stimulus]

In [9]:
lobe_data_w_vinegar.head()

,experiment_id,stimulus,valence,dorsal_to_ventral,hours_starved,order_presented,virgin,lobe,adaptation,pre_std,...,response_std,fly_id,fly_day_id,mated_single,df_mean,df_f,ΔF/F,pre_ΔF/F,greek_lobe,hours starved
0,TH-58E02_090816_Citronella,Citronella,-1,True,0,0,False,a1,0.007500,0.030948,...,0.181160,2016-08-09 00:00:000,0,NaN,0.341723,0.341723,0.341723,0.021723,α1,0
1,TH-58E02_090816_Citronella,Citronella,-1,True,0,0,False,a2,-0.010356,0.020711,...,0.157602,2016-08-09 00:00:000,0,NaN,0.230774,0.230774,0.230774,0.017802,α2,0
2,TH-58E02_090816_Citronella,Citronella,-1,True,0,0,False,a3,-0.027494,0.026490,...,0.343259,2016-08-09 00:00:000,0,NaN,1.023435,1.023435,1.023435,0.020086,α3,0
3,TH-58E02_090816_Citronella,Citronella,-1,True,0,0,False,ap1,0.026010,0.041217,...,0.258084,2016-08-09 00:00:000,0,NaN,0.476658,0.476658,0.476658,0.028166,α'1,0
4,TH-58E02_090816_Citronella,Citronella,-1,True,0,0,False,ap2,0.000798,0.041212,...,0.268255,2016-08-09 00:00:000,0,NaN,0.412485,0.412485,0.412485,0.034686,α'2,0


### SAVE THE DF

In [10]:
lobe_data_w_vinegar.to_pickle(DATA_DIR / 'siju-dataframe-unprocessed.pkl')